# Análise dos testes

### Imports

In [ ]:
from os.path import join
from json import load, dump
from copy import deepcopy

import scripts.definitions as defs
import scripts.analysis as analysis

### Configuração

In [ ]:
TEST_NAME = 'Llama-3.2-11B-Vision-Instruct_test.json'
PLOT_TITLE = 'LLaMA-3.2-11B-Vision-Instruct (Quantizado)'

### Carregamento dos testes e dados

In [ ]:
with open(join(defs.RESULTS_PATH, 'tests', TEST_NAME), 'r', encoding='utf-8') as file:
    test = load(file)

with open(join(defs.DATA_PATH, 'stt_data', 'test_dataset.json'), 'r', encoding='utf-8') as file:
    test_dataset = load(file)

with open(join(defs.DATA_PATH, 'stt_data', 'training_dataset.json'), 'r', encoding='utf-8') as file:
    training_dataset = load(file)

with open(join(defs.DATA_PATH, 'test_dataset_analysis.json'), 'r', encoding='utf-8') as file:
    test_dataset_analysis = load(file)

with open(join(defs.DATA_PATH, 'training_dataset_analysis.json'), 'r', encoding='utf-8') as file:
    training_dataset_analysis = load(file)

### Processamento dos testes

In [ ]:
sanitized_results_on_test = analysis.structure_answers(test['model']['type'], test['results_on_test'])
sanitized_results_on_training = analysis.structure_answers(test['model']['type'], test['results_on_training'])

### Salvamento dos testes processados

In [ ]:
structured_test = {
    'model_name': test['model_name'],
    'model': test['model'],
    'valid_results_on_test': [],
    'valid_results_on_training': [],
    'invalid_results_on_test': [],
    'invalid_results_on_training': []
}

valid_results_on_test = list(filter(lambda x: x['answer']['valid'], sanitized_results_on_test))
invalid_results_on_test = list(filter(lambda x: not x['answer']['valid'], sanitized_results_on_test))

print(f'Resultados válidos para testes sobre dados de teste: {len(sanitized_results_on_test) - len(invalid_results_on_test)}')
print(f'Resultados válidos para testes sobre dados de teste: {len(invalid_results_on_test)}')

valid_results_on_training = list(filter(lambda x: x['answer']['valid'], sanitized_results_on_training))
invalid_results_on_training = list(filter(lambda x: not x['answer']['valid'], sanitized_results_on_training))

print(
    f'Resultados válidos para testes sobre dados de treinamento: {len(sanitized_results_on_test) - len(invalid_results_on_training)}')
print(f'Resultados válidos para testes sobre dados de treinamento: {len(invalid_results_on_training)}')

result_lists = {
    'valid_results_on_test': valid_results_on_test,
    'valid_results_on_training': valid_results_on_training,
    'invalid_results_on_test': invalid_results_on_test,
    'invalid_results_on_training': invalid_results_on_training
}

for key, result_list in result_lists:
    for result in result_list:
        result_copy = deepcopy(result)

        del result_copy['answer']['valid']

        structured_test[key].append(result_copy)

with open(join(defs.RESULTS_PATH, 'tests', TEST_NAME.replace('.json', '_structured.json')), 'w', encoding='utf-8') as file:
    dump(structured_test, file, indent=4, ensure_ascii=False)

### Associação de pares com as respostas corretas para dados de teste

In [ ]:
# TODO: Fazer o mesmo para dados de treinamento

test_dataset_dict = {exam['exam_id']: exam for exam in test_dataset}
result_pairs = []

if test['model']['type'] == 'report':
    for result in sanitized_results_on_test:
        if result['exam_id'] in test_dataset_dict:
            result_answer = {}

            if result['answer']['valid']:
                del result['answer']['valid']
                result_answer = result['answer']
            else:
                result_answer = {
                    'elementary_lesions': ['unclear'],
                    'secondary_lesions': ['unclear'],
                    'coloration': ['unclear'],
                    'morphology': ['unclear'],
                    'size': 'unclear',
                    'skin_lesion': 'unclear',
                    'risk': 'unclear'
                }

            report = test_dataset_dict[result['exam_id']]

            expected_answer = {
                'elementary_lesions': report['elementary_lesions'],
                'secondary_lesions': report['secondary_lesions'],
                'coloration': report['coloration'],
                'morphology': report['morphology'],
                'size': report['size'],
                'skin_lesion': report['skin_lesion'],
                'risk': report['risk']
            }

            result_pair = {
                'answer': result_answer,
                'expected': expected_answer
            }

            result_pairs.append(result_pair)
else:
    for result in sanitized_results_on_test:
        if result['exam_id'] in test_dataset_dict:
            result_answer = ''

            if result['answer']['valid']:
                del result['answer']['valid']
                result_answer = result['answer']
            else:
                result_answer = 'unclear'

            expected_answer = test_dataset_dict[result['exam_id']]['skin_lesion']

            result_pair = {
                'answer': result_answer,
                'expected': expected_answer
            }

            result_pairs.append(result_pair)

### Análise

In [ ]:
 # TODO
if test['model']['type'] == 'report':
    pass
else:
    pass

# TODO: Fix
accuracy = analysis.create_confusion_matrix(pairs, model_name)